In [45]:
import librosa
import numpy as np
import time
import threading
import queue
import pickle

In [46]:
########### PARAMETERS ###########
# DO NOT MODIFY
# Desired sample rate 16000 Hz
sample_rate = 16000
# Frame length
frame_length = 512

# debug flag
DEBUG = True

In [20]:
########### AUDIO FILE ###########
# DO NOT MODIFY
# Path to the audio file
audio_file = "audio_aaico_challenge.wav"

# Read the audio file and resample it to the desired sample rate
audio_data, current_sample_rate = librosa.load(
    audio_file, 
    sr=sample_rate,
)
audio_data_int16 = (audio_data * 32767).astype(np.int16) # converting audio data from 32-bit float to 16-bit int
number_of_frames = len(audio_data_int16) // frame_length # number of frames if audio split by 512 frame length
audio_data_int16 = audio_data_int16[:number_of_frames * frame_length] # trimming audio data into multiples of 512 
audio_duration = len(audio_data_int16) / sample_rate # calculating audio duration

In [47]:
########### STREAMING SIMULATION ###########
# DO NOT MODIFY
results = np.zeros(shape=(3, len(audio_data_int16)), dtype=np.int64)
# Detection mask lines are SENT TIME, LABEL, RECEIVE TIME. 
buffer = queue.Queue()
start_event = threading.Event()

def label_samples(list_samples_id, labels):
    """
        Receives the index of samples for a frame 
        and allocates each sameple's label (0, 1) 
        and receive time.
    """
    receive_time = time.time_ns()
    results[1][list_samples_id] = labels
    results[2][list_samples_id] = receive_time

def notice_send_samples(list_samples_id):
    """
        Receives the index of samples for a frame and 
        allocates each sample's send time 
    """
    send_time = time.time_ns()
    results[0][list_samples_id] = send_time

def emit_data():
    """
        Each iteration of the loop generate a list of 
        indices of total length 512. Retreieve the frame
        from audio data using the indicies and inputting 
        into the buffer. 

        Sends the same set of indicies to "notice_send_samples" 
        to record sent time.
    """ 
    time.sleep(.5)
    print('Start emitting')
    start_event.set()
    for i in range(0, number_of_frames):
        list_samples_id = np.arange(i*frame_length, (i+1)*frame_length)
        time.sleep(frame_length / sample_rate) # Simulate real time
        frame = audio_data_int16[list_samples_id]
        
        if DEBUG:
            print(list_samples_id)
            print(frame)

        buffer.put(frame)
        notice_send_samples(list_samples_id)
    print('Stop emitting')

def process_data():
    """
        Loop runs while the current frame's processed isn't equal to
        total number of frames. 
        
        Retrieves the frame from the buffer and generates the indicies list 
        for the samples. Generates the labels for the samples, sends both 
        parameters to the label_samples function to label which samples are commands
        and which are not.
    """
    i = 0
    start_event.wait()
    print('Start processing')
    while i != number_of_frames:
        frame = buffer.get()
        ### TODO: YOUR CODE
        # MODIFY
        list_samples_id = np.arange(i*frame_length, (i+1)*frame_length)
        labels = [1 for _ in range(len(list_samples_id))]
        ###

        label_samples(list_samples_id, labels)
        i += 1
    print('Stop processing')
    
    # Save the list to a file
    with open('results.pkl', 'wb') as file:
        pickle.dump(results, file)


In [37]:
time_measurement = []

thread_process = threading.Thread(target=process_data)
thread_emit = threading.Thread(target=emit_data)

thread_process.start()
thread_emit.start()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Start emitting
Start processing
Stop emitting
Stop processing
